# DNBLab Jupyter Notebook Tutorial

## Herunterladen freier digitaler Objekte

Dieses DNBLab-Tutorial beschreibt beispielhaft, wie freie digitale Objekte der Deutschen Nationalbibliothek mit Hilfe der Metadaten eines Datensets heruntergeladen werden können. 

Entsprechende Datensets können auf mehreren Wegen zuvor heruntergeladen oder selbst erstellt werden: 

  - Vorgefertigte Datensets im DNBLab herunterladen: [www.dnb.de/dnblabsets](https://www.dnb.de/dnblabsets)
  - Datenset selbst erstellen: [DNB SRU Query Tool](https://github.com/deutsche-nationalbibliothek/SRUQueryTool/releases/tag/v1.0)
  - Tutorial zur Erstellung eigener Datensets mit Hilfe der SRU-Schnittstelle der DNB: [DNBLab-Tutorials](https://mybinder.org/v2/gh/deutsche-nationalbibliothek/dnblab/HEAD)
  - Tutorial zum Download vorgefertigte Datensets über die OAI-PMH-Schnittstellen der DNB: [DNBLab-Tutorials](https://mybinder.org/v2/gh/deutsche-nationalbibliothek/dnblab/HEAD)
  - Kleinere Datensets bis 10.000 Datensätze können zudem über den Datenshop des Portals in verschiedenen Metadatenformaten heruntergeladen werden: [Datenshop](https://portal.dnb.de/metadataShopHome)

Um den Download dazugehöriger Objekte mit diesem Tutorial durchzuführen, wählen oder erstellen Sie Ihre Datensets im Metadatenformat **MARC21-xml**. 


### Inhalt

  - Einrichten der Arbeitsumgebung
  - Benötigte Informationen aus Datenset zusammenstellen
  - Herunterladen der Objekte


**Hintergrundinformation** 

Die freien digitalen Objekte können aus dem Langzeitarchiv der DNB heruntergeladen werden. Der Zugriff auf das Objekt funktioniert hierfür mit Hilfe der jeweiligen IDN (Identifikationsnummer des Datensatzes), die als Teil eines persistenten Links den dauerhaften Zugriff auf das jeweilige Objekt ermöglicht. Im folgenden wird dieser Ansatz vorgestellt. Mehr Informationen zum Objektzugriff über Permalinks gibt es auch in unserem DNBLab-Handout: [XXXXXXXXXXXXXXXXXXXXXXXXXXXX](XXXXXX). 

Teilweise kann auf die freien digitalen Objekte auch über Links auf andere Repositorien, die sich ebenfalls in den Metadaten befinden, zugegriffen werden. Da hier allerdings große Unterschiede in der Persistenz dieser Links sowie den Zugriffsarten ( bspw. führt der Link direkt aufs Objekt oder erst auf eine Landingpage?) je nach Ablieferer bestehen, eignet sich diese Art des Zugriffs für das Herunterladen einer größerer Anzahl freier Objekte nicht. 


### Einrichten der Arbeitsumgebung

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, werden zunächst folgende benötigte Python-Bibliotheken importiert: 
Für Anfragen über die SRU-Schnittstelle wird Requests (https://docs.python-requests.org/en/latest/) verwendet, zur Verarbeitung der XML-Daten BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und etree https://docs.python.org/3/library/xml.etree.elementtree.html. Mit Pandas https://pandas.pydata.org/ werden die aus dem MARC21-Format ausgelesenen Elemente weiterverarbeitet. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden.


In [ ]:
from lxml import etree
import pandas as pd
import os
import wget
import PyPDF2
from PyPDF2 import PdfReader
import urllib.parse

### Benötigte Informationen aus Datenset zusammenstellen 

Hierfür muss zunächst das Datenset eingelesen und die benötigten Informationen extrahiert werden. Im vorliegenden Beispiel wird ein Datenset im MARC21-xml-Format verwendet. Für Datensets in anderen Metadatenformaten muss der Code entsprechend geändert werden. Liegt das Datenset bereits als CSV, Excel oder in anderer tabellarischer Form vor, kann dieser Schritt übersprungen werden. 

In [ ]:
# Laden einer MARC-xml-Datei in ElementTree - Hier den jeweiligen Dateinamen eingeben! (statt "testdata.xml")
tree = etree.parse('testdata.xml')

root = tree.getroot()                
ns = {'marc': 'http://www.loc.gov/MARC21/slim'} 
records = root.findall('.//marc:record', namespaces=ns)
print("Gefundene Records:", len(records))

Wurden die Datensätze aus dem Datenset erfolgreich geladen, kann mit Hilfe der folgenden Funktion die jeweilige IDN (Identifikationsnummer des Datensatzes) aus den Metadaten extrahiert und in ein Dataframe überführt werden: 

In [ ]:
# Funktion zum Extrahieren von Informationen, angepasst auf die IDN
def parse_record(record):
    ns = {"marc": "http://www.loc.gov/MARC21/slim"}
    
    def extract_text(xpath_query):
        fields = record.xpath(xpath_query, namespaces=ns)
        if fields:
            return "; ".join(field.text.replace('\x98', '').replace('\x9c', '') for field in fields if field.text)
        return "unknown"

    idn = extract_text("marc:controlfield[@tag='001']")

    return {"idn": idn}

In [ ]:
result = [parse_record(record) for record in records]
df = pd.DataFrame(result)
df

#### Alternative: Datenset liegt bereits im Tabellenformat vor

Liegt das Datenset bereits als Tabelle in Form einer CSV- oder Excel-Datei vor, entfallen die oberen Schritte. Stattdessen können die benötigten IDNs direkt aus der Tabelle übernommen werden. Beispielhaft folgt hier der Code für eine CSV-Datei, die eine Spalte "idn" enthält:

In [ ]:
df = pd.read_csv("metadatenset.csv", encoding="utf-8")

#### Erstellen einer Linkliste 

Nun wird für jede IDN der passende Link aufs Objekt im Langzeitarchiv der DNB zusammengesetzt: 

In [ ]:
linklist = []

for idn in df.idn: 
    link = "https://d-nb.info/" + idn + "/34"
    linklist.append(link)

In [ ]:
print(len(linklist))

### Herunterladen der freien Objekte

Für dieses Tutorial wird die Linkliste jetzt noch einmal auf die ersten 10 Einträge gekürzt und diese als `testlist` gespeichert. Sollen später alle Objekte aus allen Links in der Liste heruntergeladen werden, muss entsprechend beim Anstoßen der Funktion `download_text` die passende `linklist` übergeben werden. 

In [ ]:
testlist = linklist[:10]
print(testlist)

Nun wird die Funktion zum Herunterladen der Dateien definiert. Diese erstellt zunächst ein neues Verzeichnis mit Wunschnamen (`save_directory`) für die gesammelten Dateien, lädt diese dann herunter und speichert die Objekte unter Nutzung der IDN als Dateiname: 

In [ ]:
import requests

def download_text(testlist, save_directory):
    # Erstellen des später definierten Speicherverzeichnis, falls es dieses noch nicht gibt
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    for link in testlist: 
        # IDN ermitteln:
        idn = link.split('/')[-2]
        # Original-Dateiname und -Endung aus Content-Disposition header ermitteln und als Basis nutzen:
        response = requests.head(link)
        content_disposition = response.headers.get('Content-Disposition')
        if content_disposition:
            orig_filename = content_disposition.split('filename=')[-1].strip('";')

            # Dateiname aus IDN und originalem Dateiname zusammensetzen: 
            file_name = f"{idn}_{orig_filename}"
            file_path = os.path.join(save_directory, file_name)
            print(f" Starte Download: {file_name}")
            try: 
                #Datei herunterladen:
                wget.download(link, out=file_path)
            except Exception as e:
                print(f"Fehler beim Herunterladen von {link}: {e}")

        else: 
            print(f"Kein Objekt zum Datensatz mit der IDN {idn} gefunden. Lade nächstes Objekt...") 

    print("Fertig.")

Hier wird nun der Name für das gewünschte Verzeichnis benannt und anschließend die Funktion gestartet und ihr die gewünschte Linkliste übergeben (in diesem Fall die `testlist`): 

In [ ]:
save_directory = "downloads"

download_text(testlist, save_directory)